In the Beginning:

- There was simplification.
- Brute Equational Search

Greedy:
- Missed optimizations
- Ordering of rules matters
- Hard to reason about
- Nontermination

Problems:
- Brute search is very explosive

Term Orderings
- Mostly getting a smaller term is better
- If terms keep shrinking, they have to stop.
- Ground KBO


Completion
- Fact: Ground completion terminates

Canonized Egraph = Canoninized GRS
- We have terms
- 

"Skolemization" of eclasses


# Tree Automata
- Strings ~ single argument terms
- Term rewriting formulation of DFA
- Generalizes to Tree

# Resolution
- Resolution
- Ordered Resolution
- Datalog
- Prolog

note: have box that like fills in? Left side is propositional, right is equational. 
not a \/ b === a -> b

# Strategies
- Set of Support
- UR-resolution

# Equations
- paramodulation
- Superposition
- Egglog
- Functional Logic Programming

# Philosophy
- What is it about egg? Pictures + Compiler. Not a conceptual high bar to entry.
- Non completeness
- Limited scope leads to highly refined, fast, implementation


- Handbook of automated reasononig
- Term Rewriting and All That
- Automated Reasoning: An Introduction

I've noted before it isn't obvious what goes into the box next to completion in the propositional case

Ground atomic superposition is a contextual union find.


? = completion
lambda prolog = ?
hilog = ?
contextual datalog = ?
hypothetical prolog = ?



In [ ]:
def simplify1(t : Term) -> Term:


# Bits and Bobbles

Slides from juptyer?  I can nbconvert


Animations?